In [ ]:
from pathlib import Path
import os

import pandas as pd

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

***Notebooks are written for Jupyter and might not display well in Gitlab***

# Load experimental data

First, we load the data that were cleaned in the previous chapter _"CH1_Etics_Data_cleaning"_
The measurements are loaded using pandas <code>DataFrame</code>

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "ressources/study_df.csv",
    index_col=0,
    parse_dates=True
)

# Physical model

For this example we propose a resistance/capacity approach.
 Based on electrical circuit analogy, each layer of the wall is modeled by two resistance and a capacity:

<img src="images/Wall_model.png"  height="300">

The following is a brief description of the thermal model, as it is not the scope of this document. See the <code>*mo</code> for full informations.

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - Resistances : $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $
    - Capacity : $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $


- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.


- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $
    - With the surrounding considered to be at $ T_{ext} $


- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.


- Temperatures $ T_{ext}$ and $T_{int} $ are boundary conditions


Initial conditions for the layers temperatures are taken from the measured data.
 It is assumed to be the mean temperature measured by the sensors on each face of a layer.
 In python and using modelica "object name", it can be written :

In [ ]:
init_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
}

We specify the simulation running options. As the initial condition, it is written
as a python dictionary.

In Modelica, <code>startTime</code> and <code>stopTime</code> correspond to the number
of seconds since the beginning of the year. The values can be found in the file created
earlier using <code>df_to_combitimetable</code>

Another way is to use the index of the <code>DataFrame</code> we just created.
The modelitool function <code>modelitool.combitabconvert.datetime_to_seconds</code>
helps you convert datetime index in seconds.

For the sensitivity Analysis we will just use the 2 first day 22/03 and 23/03

<code>stepSize</code> is the simulation timestep size. In this case it's 5min or
300sec.

<code>tolerance</code> and <code>solver</code> are related to solver configuration
do not change if you don't need to.

In [ ]:
from modelitool.combitabconvert import datetime_to_seconds

In [ ]:
second_index = datetime_to_seconds(reference_df.loc["2018-03-22":"2018-03-23"].index)
simulation_opt = {
        "startTime": second_index[0],
        "stopTime": second_index[-1],
        "stepSize": 300,
        "tolerance": 1e-06,
        "solver": "dassl"
}

We can now define a modelitool <code>Simulator</code>. This object is designed
to handle modelica simulation and output post treatment. It will be used
in objects that automate simulation such as an <code>Identificator</code>
or a <code>SAnalysis</code> object.

In [ ]:
from modelitool.simulate import Simulator

In [ ]:
# Values in output list correspond to sensors name and value "T"
simu = Simulator(
    model_path=Path(TUTORIAL_DIR) / "ressources/etics_v0.mo",
    simulation_options=simulation_opt,
    init_parameters=init_dict,
    boundary_df=reference_df,
    output_list=["T_coat_ins.T",
                 "T_ins_ins.T",
                 "Tw_out.T"],
)

From here, it is very simple to run a simulation using <code>simulate()</code>
method, and to get the results required in <code>output_list</code> using
<code>get_results()</code> method.

In [ ]:
simu.simulate()
initial_results = simu.get_results()

*Important Note: Temperature outputs of the modelica model are in Kelvin.
We manually add 273.15°C to compare results or to compare measure and model outputs*


In [ ]:
# Plotly lines
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=initial_results.index,
    y=initial_results["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x=initial_results.index,
    y=reference_df.T_Ins_Ins + 273.15,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Model VS Reality : temperature between two layer of insulation',
    xaxis_title='Time [sec since 01/01]',
    yaxis_title='Temperature [K]')

fig.show()

Considering the above graphic, we could say that the model results are pretty bad.
- The difference between predicted and measured temperature reaches ~10K
- There seem to be a small "shift" between model and reality.
Reference temperature peaks are happening earlier than model peaks.

There can be a lot of causes to this discrepancy. From physical phenomenon approximation
to material physical properties values.

*A good Idea would be to plot the other outputs variables or to perform a heat
balance analysis. But remember that this is a tutorial to use modelitool :).*

*For now we will just go on, and perform a sensitivity analysis*

# Sensitivity analisys

This project aims at characterizing the insulation material thermal conductivity.
To do so, we want to find the value $ \lambda_{etics} $ that would minimise the discrepancy between
predicted temperature between 2 layers of insulation material and the sensors measures.

However, it is very important to know how $ \lambda_{etics} $ is important
to describe the error. Other assumptions, such as material thermal properties may
have a strong influence on the model prediction.

Therefore, we use the sensitivity analysis to "rank" the parameter by order of influence
on the error between measured temperature and model prediction.

The chosen error function is the Mean Square Error (RMSE):

$$
MSE = \frac{1}{1-N}\sum \limits_{i=1}^{N} (y_i - \hat y_i)^2
$$

The chosen parameters for the sensitivity analysis are listed below:
- Concrete thermal capacity <code>capa_concrete</code> with an uncertainty of 20%
- Concrete density <code>rho_concrete</code> with an uncertainty of 20%
- All layer thermal conductivity with an uncertainty af 20%
- The coating shortwave solar absorption coefficient <code>Alpha_clo</code>
with an uncertainty of 20%
- The inside and outside conductive/convective equivalent thermal resistance
with an uncertainty of 20%

In modelitool, these uncertainties must be described using a list of dictionaries:

In [ ]:
modelitool_problem = [
    {
        'name': 'capa_concrete',
        'interval': (1000-0.2*1000, 1000+0.2*1000),
        'init': 0,
        'type': "Real"
    },
    {
        'name': 'rho_concrete',
        'interval': (875-0.2*875, 875+0.2*875),
        'init': 0,
        'type': "Real"
    },
   {
        'name': 'lambda_concrete',
        'interval': (1-0.2*1, 1+0.2*1),
        'init': 0,
        'type': "Real"
    },  
   {
        'name': 'lambda_coating',
        'interval': (1-0.2*1, 1+0.2*1),
        'init': 0,
        'type': "Real"
    },    
    {
        'name': 'Lambda_ins.k',
        'interval': (0.04-0.2*0.04, 0.04+0.2*0.04),
        'init': 0,
        'type': "Real"
    },    
    {
        'name': 'Alpha_clo.k',
        'interval': (0.5-0.2*0.5, 0.5+0.2*0.5),
        'init': 0,
        'type': "Real"
    },     
    {
        'name': 'R_conv_ext.k',
        'interval': (0.04-0.2*0.04, 0.04+0.2*0.04),
        'init': 0,
        'type': "Real"
    },
    {
        'name': 'R_conv_int.k',
        'interval': (0.13-0.2*0.13, 0.13+0.2*0.13),
        'init': 0,
        'type': "Real"
    }   
]

We can now use a <code>SAnalysis</code> to set-up the study. We have to pass
the <code>Simulator</code> previously describe, along with the corresponding
 problem description. A Sensitivity Analysis is also required. In this case we choose Sobol
, as there is few uncertain parameter.

*Note: for now only <code>Sobol</code>, <code>FAST</code>, <code>RBD_fast</code>, 
and <code>Morris</code> methods are implemented.*

In [ ]:
from modelitool.sensitivity import SAnalysis

sa_study = SAnalysis(
    simulator=simu,
    sensitivity_method="Sobol",
    parameters_config=modelitool_problem
)

We draw a sample of parameters to simulate. Each method has its sampling method.
Please see SALib documentation for further explanation (https://salib.readthedocs.io/en/latest/index.html)

In [ ]:
# Additional arguments can be passed. if arguments is not used it
# Uses default SALib configuration
sa_study.draw_sample(n=100, arguments={"calc_second_order": True})

The sample is available as a 2d array <code>sa_study.sample</code>. Lines are simulations
to run and columns are parameters values.

Let's run the simulations.

**CAREFUL depending on your computer, it can take a long time (up to 30')**

In [ ]:
sa_study.run_simulations()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
sa_study.analyze(
    aggregation_method=mean_squared_error,
    indicator="T_ins_ins.T",
    reference=reference_df.loc["2018-03-22":"2018-03-23", "T_Ins_Ins"] + 273.15
)

We can now have a look at the sensitivity analysis results.
They are stored in <code>sensitivity_results</code>. It holds the output formatted
by <code>SALib</code>. It is possible to get it as a <code>DataFrame</code>
using <code>to_df()</code> (see the doc).

First, let's sum the indices of Total order.

In [ ]:
sum_st = sa_study.sensitivity_results.to_df()[0].sum().loc["ST"]
mean_conf = sa_study.sensitivity_results.to_df()[0].mean().loc["ST_conf"]

print(
    f"The sum of Sobol Total or index is {sum_st} \n"
    f"The mean confidence interval is {mean_conf}"
)

The sum of all the indices is very close to 1. Also, the mean confidence interval
seems to be very low. Results of the sensitivity analysis appear to be robust.

We can now plot the results

In [ ]:
sa_study.plot(kind="bar")

- Fortunately $\lambda_{etics}$ seems to be the most influential parameter on the error between
model outputs and sensor measure
- Two other parameters have a strong influence on the error :
    - the coating coefficient of absorption (solar radiation) $\alpha_{coat}$
    - the outside conduction/convection resistance
- Looking at the interval of confidence, this ranking is reliable

To understand the possible effects of interactions between parameter. We can
have a look at the indices of order 1 an 2

In [ ]:
sum_s1 = sa_study.sensitivity_results.to_df()[1].sum().loc["S1"]
mean_conf1 = sa_study.sensitivity_results.to_df()[1].mean().loc["S1_conf"]
sum_s2 = sa_study.sensitivity_results.to_df()[2].sum().loc["S2"]
mean_conf2 = sa_study.sensitivity_results.to_df()[2].mean().loc["S2_conf"]

print(
    f"The sum of S1 index is {sum_s1} \n"
    f"The sum of S2 index is {sum_s2} \n"
    f"The S1 mean confidence interval is {mean_conf1}\n"
    f"The S2 mean confidence interval is {mean_conf2}"
)

These results are not very good (remember $ S_1 + S_2 + ... + S_n = 1 $)

But it means that parameters are influential at the 1st order. The interaction effect
between the parameters is negligible.

# Morris method
If you chose Morris method, re-formatted figure for  results is available using <code>plot_morris_scatter</code>.

In [ ]:
from modelitool.sensitivity import plot_morris_scatter 
plot_morris_scatter(salib_res=sa_study.sensitivity_results, title='Elementary effects', unit='J', autosize=True) 

# Conclusion on sensitivity analysis

The sensitivity analysis allows us to rank the influence of uncertain parameter
on an indicator. In this case we choose the $MSE$ between model output
and measurement.

Fortunately it shows that the most influential parameters was the insulation thermal
conductivity. But we can't discard the coefficient of absorption of solar radiation
that accounts for 30% of the variance of the error. Also, the thermal resistance that models
the conductive/convective heat transfer is not negligible (~8% of the variance).

In the following chapter, we will see how to use modelitool to identify the
optimal values for these parameters in order to fit the measurement.

We will also assess the reliability of the obtained values

# Annex: Going further

More information could be obtained from the results the sensitivity analysis sample.
It would be a shame to throw away all this computational time.

## Dynamic sensitivity Analysis
Parameters may not have a constant impact on the error.
For example the solar absorption coefficient will probably have a stronger impact
during day time.

By default, indices are “relative” (normalised by the total variance at each time step). For unnormalised sensitivity indices, giving information on the “absolute” influence of each parameter, set the argument <code>absolute</code> to <code>True</code> in the <code>SAnalysis</code> (<code>False</code> by default).

Below is the calculation of Sobol Indexes on the insulation mean temperature

In [ ]:
import numpy as np

sa_study.dynanalyze(
    aggregation_method=np.mean,
    indicator="T_ins_ins.T",
    freq="15T",
    absolute=False
)

In [ ]:
sa_study.plot(
    kind="dynamic_ST",
    arguments=dict(
        indicator="T_ins_ins.T",
        y_label="Sobol Total index",
        title="Dynamic Sobol Total index Insulation Mean temperature"
    )
)

## Parallel plot
Parallel coordinate plot can be very usefully to represent the "interactions" between one (or several) observable outputs
and the value of the parameters that cause it. Contrary to scatter plot, it can have a lot of dimensions.
Finally, it is interactive. You can select several simulation configurations of interest.

To configure the plot, you have to specify the "outputs of interest"
For example, in a calibration workflow, it could be an indicator of the gap between the model output
and the measured value.

In the configuration below, we propose 3 indicators:
- Normalise mean biased error between measured and modeled insulation temperature
- Coefficient of variation of the root mean squared error between measured and modeled insulation temperature
- Coefficient of variation of the root mean squared error between measured and modeled coating temperature

In [ ]:
from modelitool.metrics import  nmbe
from modelitool.metrics import cv_rmse

args = {
    "indicator_dict_list": [
        {
            "name": "nmbe_Tins_ins",
            "aggregation_method": nmbe,
            "indicator": "T_ins_ins.T",
            "ref": reference_df.loc["2018-03-22":"2018-03-23", "T_Ins_Ins"] + 273.15,
        },
        {
            "name": "cv_rmse_Tins_ins",
            "aggregation_method": cv_rmse,
            "indicator": "T_ins_ins.T",
            "ref": reference_df.loc["2018-03-22":"2018-03-23", "T_Ins_Ins"] + 273.15,
        },
        {
            "name": "cv_rmse_T_Ins_Coat",
            "aggregation_method": cv_rmse,
            "indicator": "T_coat_ins.T",
            "ref": reference_df.loc["2018-03-22":"2018-03-23", "T_Ins_Coat"] + 273.15,
        },
    ],
    "plot_options": {
        "colorby": "cv_rmse_Tins_ins"
    }
}

In [ ]:
sa_study.plot(
    kind='parallel',
    arguments=args
)

## Sample plot

The last available simply plots all the simulation results on a single graph, and allows you to compare it
to a reference (the measurement for example).
You can use this plot to assess the validity of your parameters boundaries. If the reference value is not "contained" in your sample,
there is a great chance that the truth value of some parameters lie outside the boundaries, or that your model is missing a physical phenomenon.

***Careful heavy sample may cause trouble to display***

In [ ]:
sa_study.plot(
    kind='sample',
    arguments=dict(
        indicator= "T_ins_ins.T",
        ref= reference_df.loc["2018-03-22":"2018-03-23", "T_Ins_Ins"] + 273.15,
        title= "Model sample results VS Measurement",
        y_label= "temperature [K]"
    )
)